# Churn Rate 2: Bootstrap

We are working in Online Learning platform and working on developing Churn Rate prediction model. We want to know which user have higher probability to leave our platform: starts to visit the website rarely, or he/she does not know about all programs available.

We carefully study what problems our users have. We make interviews with users asking what they like in the platform and what can be added or fixed.

In the Churn Rate part 1 task we completed data aquisition, features engineering and model development

In this task we will do advanced validation of the model and metrics calculations.

# Boostrapped ROC-AUC

When we developed good (as we think) model - for example with ROC-AUC = 93%  
We should also think: How we are sure in our evaluation?  
How we can know that in nex month, when we again calculate metric, ROC-AUC will be 95% and not 76%?  
We really want to know what to expect from the model.  

Obviously model performance will fluctuate, but in what extend?

To answer this question, we need to calculate lower and upper boundaries of the **Confidence Interval**

![](./pic/1.png)

Every metric (and ROC-AUC is not an exclusion) - is a statistics, which depends on dataset on which we calculate it.  
In our task - our dataset is our users, which we include in our test dataset.  
With time our users change their behavior and therefore features are also changing.

## Confidence Intervals  
- 95% Confidence Interval - values diapason of some statistical measure in 95% of cases. Usually, centered interval is considered. Therefore, on the left side and on the right side will be 2.5% of the values which are not included in Confidence Interval.
- Confidence Bounds - bounds of confidence intervals
- LCB (Lower Confidence Bound) - lower boundary of the confidence interval
- UCB (Upper Confidence Bound) - upper boundary of the confidence interval

How we can calculate Confidence Interval for ROC-AUC?

Imagine: you remember only formulas for Normal distribution, but how we can implement them to our task?  
You start Google: "How to calculate Confidence Interval for ROC-AUC". Find some scary looking formulas and do not understand what is happening there :)  
![](./pic/2.png)

Calm down, there is one secret key, how we can start doing this task:

# Empirical distribution function:  
Our general dataset - all users in every moment in future. Ideally, we would simply generate subsets from this general dataset, calculate ROC-AUC on each of them, take Quantiles, which represent LCB and UCB.

Unfortunately, we do not have access to the general dataset.  
We have only one subset of data on which we can build only empirical distribution function.  
We can take subset and generate 10 000 speudo subsets.

# Bootstrap

This method is called "Bootstrap" - it allows us based on our subset imitate generation of other similar subsets. In suits our goal of calculating Confidence Interval for ROC-AUC metrics.

![](./pic/3.png)

we need to create python function for evaluating Confidence Interval for ROC-AUC by bootstrap.  
inputs - trained classifier (from sklearn library), test subset, and size of confidence interval and number of boostrap subsets